## Extended Kalman Filter
I decided to implement this first in Python instead of just filling in the blanks and patching together the boilerplate code from Udacity.

This will allow me to code it up from scratch and really get an intimate understanding of Kalman Filters before I complete the implementation in C++.

Using a Jupyter notebook will also help me to take thorough notes on the math in the midst of the code.

### Kalman Filter
The first step will be to recreate the normal Kalman Filter implementation that I already built in C++ during the lessons.

In [121]:
import numpy as np

In [122]:
class KalmanFilter():
    """Tracks the state."""
    
    def __init__(self):
#   VectorXd x_;

        self.x_ = None
#   // state covariance matrix
#   MatrixXd P_;
        self.P_ = np.matrix(np.ndarray((4,4)))
#   // state transistion matrix
#   MatrixXd F_;
        self.F_ = np.matrix([1, 0, 1, 0,
                             0, 1, 0, 1,
                             0, 0, 1, 0,
                             0, 0, 0, 1]).reshape(4,4)
#   // measurement matrix
#   MatrixXd H_;
        self.H_ = np.matrix([1, 0, 0, 0,
                             0, 1, 0, 0]).reshape(2,4)
#   // measurement covariance matrix
#   MatrixXd R_;
        self.R_ = np.matrix([0.0225, 0,
                             0, 0.0225]).reshape(2,2)
    def Predict(self):
        self.x_ = self.F_ * self.x_
        Ft = self.F_.transpose()
        self.P_ = self.F_ * self.P_ * Ft + self.Q_
        
    def Update(self, z):
#         print "Shape of z", z.shape()

        z_pred = self.H_ * self.x_

        z = np.matrix(z).reshape(2,1)
        y = z - z_pred
        Ht = self.H_.transpose()
        S = self.H_ * self.P_ * Ht + self.R_
        Si = S.I
        PHt = self.P_ * Ht
        K = PHt * Si

#       // new estimate

        self.x_ = self.x_ + (K * y)

        # Don't know why but x_ keeps getting assigned a datatype and it prevents matrix mult.
        self.x_.dtype = None
        x_size = len(self.x_)
        I = np.identity(x_size)

        self.P_ = (I - K * self.H_) * self.P_

In [127]:
class Tracking():
    """Tracks the state of the Kalman Filter."""
    def __init__(self):
        self.kf_ = KalmanFilter()
        self.previous_timestamp_ = 0
        self.noise_ax = 5
        self.noise_ay = 5
        self.is_initialized_ = False
    def ProcessMeasurement(self, measurement_pack):
        if not self.is_initialized_:
            print( "Kalman Filter Initialization " )
            # Handle Lidar and Radar differently.
            if measurement_pack.sensor_type_ == "L":
                self.kf_.x_ = np.matrix([measurement_pack.raw_measurements_[0],
                          measurement_pack.raw_measurements_[1],
                          0, 0]).reshape(4,1)
                self.kf_.x_.dtype = None                
            elif measurement_pack.sensor_type_ == "R":
                ro = 
                self.kf_.x_ = np.matrix([measurement_pack.raw_measurements_[0],
                          measurement_pack.raw_measurements_[1],
                          0, 0]).reshape(4,1)
                self.kf_.x_.dtype = None
                
            self.previous_timestamp_ = measurement_pack.timestamp_
            
            
            self.is_initialized_ = True
            return
        
        dt = (measurement_pack.timestamp_ - self.previous_timestamp_) / 1000000.0
        self.previous_timestamp_ = measurement_pack.timestamp_
        print("Time: " + str(dt))
#         // 1. Modify the F matrix so that the time is integrated
        self.kf_.F_[0, 2] = dt;
        self.kf_.F_[1, 3] = dt
#         // 2. Set the process covariance matrix Q
        self.kf_.Q_ = np.matrix([(dt**4)*self.noise_ax/4, 0, (dt**3)*self.noise_ax/2, 0,
                                 0, (dt**4)*self.noise_ay/4, 0, (dt**3)*self.noise_ay/2,
                                 (dt**3)*self.noise_ax/2, 0, (dt**2)*self.noise_ax/1, 0,
                                 0, (dt**3)*self.noise_ay/2, 0, (dt**2)*self.noise_ay/1]).reshape(4,4)

#         // 3. Call the Kalman Filter predict() function
        self.kf_.Predict()
#         // 4. Call the Kalman Filter update() function
        self.kf_.Update(measurement_pack.raw_measurements_)
        
        
        
        print("x_= " )
        print(self.kf_.x_)
        print("P_= ")
        print(self.kf_.P_)

## Read the measurement file


In [128]:
class MeasurementPackage():
    def __init__(self):
        self.sensor_type_ = ""
        self.timestamp_ = 0
        self.raw_measurements_ = None
        
class GroundTruthPackage():
    def __init__(self):
        self.sensor_type_ = ""
        self.timestamp_ = 0
        self.gt_values_ = None

In [129]:
#//input file with laser and radar measurements
in_file_name_ = "test.txt"

#//output file with the estimation (px, py, vx, vy), measurements (px and py meas) and ground truths (px, py, vx, vy)
out_file_name_ = "obj_pose-laser-radar-output.txt"

with open(in_file_name_, 'r') as f:
    data_lines = f.readlines()

measurement_pack_list = []
gt_pack_list = []

# Extract data from line

for line in data_lines:
    entry = line.split("\t")
    meas_package =  MeasurementPackage()
    gt_package = GroundTruthPackage()
    sensor_type = entry.pop(0)
    if sensor_type == "L":
        # Read Measurements
        meas_package.sensor_type_ = sensor_type
        x = entry.pop(0)
        y = entry.pop(0)
        meas_package.raw_measurements_ = (float(x), float(y))
        timestamp = entry.pop(0)
        meas_package.timestamp_ = float(timestamp)
        measurement_pack_list.append(meas_package)
    elif sensor_type == "R":
        meas_package.sensor_type_ = sensor_type
        ro = entry.pop(0)
        phi = entry.pop(0)
        ro_dot = entry.pop(0)
        meas_package.raw_measurements_ = map(float,(ro, phi, ro_dot))
        timestamp = entry.pop(0)
        meas_package.timestamp_ = float(timestamp)
        measurement_pack_list.append(meas_package)
        
    # Get the ground truth package
    
    x_gt, y_gt, vx_gt, vy_gt = map(float, entry)
    gt_matrix = np.matrix([x_gt, y_gt, vx_gt, vy_gt]).reshape(4,1)
    gt_package.gt_values_ = gt_matrix
    
    gt_pack_list.append(gt_package)
    


In [130]:
tracking = Tracking()
estimations = []
for measurement, ground_truth in zip(measurement_pack_list, gt_pack_list):
    tracking.ProcessMeasurement(measurement)
    estimations.append(tracking.kf_.x_)
    

Kalman Filter Initialization 
Time: 0.049996


TypeError: unsupported operand type(s) for *: 'int' and 'NoneType'

In [134]:
t = measurement_pack_list[0]

In [137]:
t.timestamp_

1477010443399637.0

In [112]:
# Calculate RMSE

def CalculateRMSE(estimations, ground_truth):
    rmse = np.matrix([0.,0.,0.,0.]).reshape(4,1)

    if len(estimations) != len(ground_truth) or len(estimations) == 0:
        print "Not right size, too bad"
        return rmse

    for estimation, truth in zip(estimations, ground_truth):
        residual = estimation - truth.gt_values_
        
        residual = np.square(residual)
        rmse += residual
        
        #     Calculate the meain
        rmse = rmse/len(estimations)
        # Calculate the sqrt
        rmse = np.sqrt(rmse)

    return rmse

rmse = CalculateRMSE(estimations, gt_pack_list)

In [118]:
rmse

matrix([[ 0.06399414],
        [ 0.10580425],
        [ 0.883126  ],
        [ 3.96462076]])

In [120]:
gt_package.gt_values_

matrix([[ 11.35    ],
        [ -1.85    ],
        [  0.909847],
        [  2.72953 ]])

In [95]:
estimations[9]

matrix([[ 8.10840938],
        [ 0.13600664],
        [-6.44629982],
        [-2.18989325]])

In [38]:

tracking.kf_.F_ * tracking.kf_.x_

matrix([[  8.44822240e+00,   8.40270263e+00],
        [  2.97086340e-01,   2.51566564e-01],
        [  8.47067971e-04,  -9.08511776e-01],
        [  9.09629817e-01,   2.70973052e-04]])

In [43]:
tracking.kf_.H_ * tracking.kf_.x_ # z_pred

matrix([[ 8.4482224 ,  8.40270263],
        [ 0.29708634,  0.25156656]])

In [45]:
measurement.raw_measurements_

(8.45582, 0.253997)

In [47]:
tracking.kf_.H_

matrix([[1, 0, 0, 0],
        [0, 1, 0, 0]])

In [ ]:
# Create a tracking instance
tracking = Tracking()

In [126]:
gt_package.gt_values_

['11.35', '-1.85', '0.909847', '2.72953']

In [85]:
test = test.split("\t")

In [89]:
entry = test
entry

['R',
 '8.46642',
 '0.0287602',
 '-3.04035',
 '1477010443399637',
 '8.6',
 '0.25',
 '-3.00029',
 '0\n']

In [91]:
sensor_type = entry.pop(0)

In [94]:
if sensor_type == 'R':
    meas_package.sensor_type = 

['8.46642',
 '0.0287602',
 '-3.04035',
 '1477010443399637',
 '8.6',
 '0.25',
 '-3.00029',
 '0\n']